# CUDA を使って画像に3DLUTをかける

## 3DLUT の補間処理の元ネタ

FUJIFILM社の「LOGとLUTの基礎講座」資料 P.38 のアルゴリズムを使用する。

![figure1](./figure/3dlut_interpolation.jpg)

リンク：[LOGとLUTの基礎講座](http://fujifilm.jp/business/broadcastcinema/solution/color_management/is-mini/promotion/pack/pdf/news/20150122_jppa.pdf)

## Python向けに補間処理の数式を考える。

![figure1](./figure/sekkei.jpg)

8頂点で囲われた立方体を上図を参考に8領域に分ける。それぞれの領域の体積を$V_0 \sim V_7$ と置くと、以下の式で算出できる。


\begin{align}
V_0 & = R_0 \cdot G_0 \cdot B_0 \\
V_1 & = R_0 \cdot G_0 \cdot B_1 \\
V_2 & = R_0 \cdot G_1 \cdot B_0 \\
V_3 & = R_0 \cdot G_1 \cdot B_1 \\
V_4 & = R_1 \cdot G_0 \cdot B_0 \\
V_5 & = R_1 \cdot G_0 \cdot B_1 \\
V_6 & = R_1 \cdot G_1 \cdot B_0 \\
V_7 & = R_1 \cdot G_1 \cdot B_1 \\
\end{align}

\begin{align}
R_{out} & = V_0 \cdot L_0[R] + V_1 \cdot L_1[R] + V_2 \cdot L_2[R] + V_3 \cdot L_3[R] + V_4 \cdot L_4[R] + V_5 \cdot L_5[R] + V_6 \cdot L_6[R] + V_7 \cdot L_7[R] \\
G_{out} & = V_0 \cdot L_0[G] + V_1 \cdot L_1[G] + V_2 \cdot L_2[G] + V_3 \cdot L_3[G] + V_4 \cdot L_4[G] + V_5 \cdot L_5[G] + V_6 \cdot L_6[G] + V_7 \cdot L_7[G] \\
B_{out} & = V_0 \cdot L_0[B] + V_1 \cdot L_1[B] + V_2 \cdot L_2[B] + V_3 \cdot L_3[B] + V_4 \cdot L_4[B] + V_5 \cdot L_5[B] + V_6 \cdot L_6[B] + V_7 \cdot L_7[B]
\end{align}